In [ ]:
import cv2 as cv2
import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras import optimizers, initializers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten,Dropout, MaxPooling2D, AveragePooling2D
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.callbacks import EarlyStopping, History
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

import import_ipynb
import dataSetUtility as dsu
import plottingUtility as pltu

In [15]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from keras.utils import np_utils
from tensorflow.keras.applications import VGG16, imagenet_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import GlobalAveragePooling2D

In [16]:
import imageio
from PIL import Image
import tifffile

In [17]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import seaborn as sn
import numpy as np
from scipy import ndimage

import random
import PIL
import os
import cv2 as cv2
import itertools
import glob

In [18]:
#path to the directories
pathTrain=r'C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train'
pathTest=r'C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\test'

labels = [os.path.basename(i) for i in glob.glob(pathTrain + '/*', recursive=True)]
numberOfClasses = len(labels)
print("Class number: ", numberOfClasses)
print("Class names: ", labels)

Class number:  15
Class names:  ['Bedroom', 'Coast', 'Forest', 'Highway', 'Industrial', 'InsideCity', 'Kitchen', 'LivingRoom', 'Mountain', 'Office', 'OpenCountry', 'Store', 'Street', 'Suburb', 'TallBuilding']


In [19]:
def get_random_crop(img, width, height, dim=64):
    assert img.shape[0] >= height
    assert img.shape[1] >= width
    x = random.randint(0, img.shape[1] - width)
    y = random.randint(0, img.shape[0] - height)
    img = img[y:y+height, x:x+width]
    resImg = cv2.resize(img, (dim,dim), interpolation=cv2.INTER_CUBIC)
    return resImg

In [20]:
import math
import copy

def warpAffine(src, M, dsize, from_bounding_box_only=False):
    """
    Applies cv2 warpAffine, marking transparency if bounding box only
    The last of the 4 channels is merely a marker. It does not specify opacity in the usual way.
    """
    return cv2.warpAffine(src, M, dsize)

def rotate_image(image, dim=64):
    """Rotate the image counterclockwise.
    Rotate the image such that the rotated image is enclosed inside the
    tightest rectangle. The area not occupied by the pixels of the original
    image is colored black.
    Parameters
    ----------
    image : numpy.ndarray
        numpy image
    angle : float
        angle by which the image is to be rotated. Positive angle is
        counterclockwise.
    Returns
    -------
    numpy.ndarray
        Rotated Image
    """
    angle = random.randint(0, 360)
    # get dims, find center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)

    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    # perform the actual rotation and return the image
    image = warpAffine(image, M, (nW, nH), False)

    resImg = cv2.resize(image, (dim,dim), interpolation=cv2.INTER_CUBIC)

    return resImg

In [21]:
def dataAugmentationWithCropRot(path, labels, dimension=64, d=1):
    x = [] #New ImageSet
    y = [] #New Label
    for index,name in enumerate(labels):
        s = path+'/'+name+'/{}'
        s2 = path+'/'+name+'/'
        temp = [s.format(i) for i in os.listdir(path+'/'+name+'/')]
        q=0
        for image in temp:
            q = q+1
            tmp = cv2.resize(cv2.imread(image,cv2.IMREAD_GRAYSCALE),(dimension,dimension),interpolation=cv2.INTER_CUBIC)
            x.append(tmp)
            imageio.imwrite(s2 + 'outfile1' + str(q) + str(index) + '.jpg', tmp)
            q = q+1
            x.append(cv2.flip(tmp,1))
            imageio.imwrite(s2+ 'outfile2' + str(q)  + str(index)+  '.jpg', cv2.flip(tmp,1))
            q = q+1
            x.append(get_random_crop(tmp, dimension-24, dimension-24, dimension))
            imageio.imwrite(s2+ 'outfile3' + str(q)+   str(index) + '.jpg', get_random_crop(tmp, dimension-24, dimension-24, dimension))
            q = q+1
            x.append(rotate_image(tmp, dimension))
            imageio.imwrite(s2+ 'outfile4' + str(q) + str(index)+ '.jpg', rotate_image(tmp, dimension))
            q = q+1
            y.append(index)
            y.append(index)
            y.append(index)
            y.append(index)
        
    x = np.asarray([np.reshape(im, (dimension, dimension, d)) for im in x])
    x=x/255
    y = np.asarray(y)
    return x,y

In [22]:
#xTrain, yTrain = dataAugmentationWithCropRot(pathTrain, labels, dimension=224)

In [23]:
#xTrain.shape

In [24]:
xTrain, yTrain = dataAugmentationWithCropRot(pathTrain, labels, dimension=224, d=1)
xTest, yTest = dsu.loadImages(pathTest, labels, dimension=224)

In [25]:
yTrainDummy = to_categorical(yTrain, 15)
yTestDummy = to_categorical(yTest, 15)

In [26]:
model = VGG16(weights='imagenet', include_top=False)
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [27]:
def extractFeatures(model, path, labels):
    
    pathImages = []
    for label in labels:
        s = path+'/'+label+'/{}'
        temp = [s.format(i) for i in os.listdir(path+'/'+label+'/')]
        pathImages = pathImages + temp
    
    res = []
    count = 0
    for p in pathImages:
        count += 1
        img = load_img(p, target_size=(224,224))
        print(p)
        img = img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = imagenet_utils.preprocess_input(img)
        res.append(img)
        
    images = np.vstack(res)
    features = model.predict(images, batch_size=64)
    featuresFlatten = features.reshape((features.shape[0], 7 * 7 * 512))
    
    return images, features, featuresFlatten

In [ ]:
trainImages, featureMatrix, featureArray = extractFeatures(model, pathTrain, labels)

C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/image_0001.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/image_0002.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/image_0005.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/image_0009.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/image_0010.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/image_0012.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/image_0015.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/image_0022.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/image_0023.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet

C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile11360.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile11410.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile11460.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile11510.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile11560.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile1160.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile11610.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile11660.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile11710.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2

C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile160.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile1610.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile1660.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile1710.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile1760.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile1810.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile1860.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile1910.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile1960.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\

C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile24220.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile24270.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile24320.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile24370.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile24420.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile24470.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile24520.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile24570.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile24620.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-

C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile33730.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile33780.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile3380.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile33830.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile33880.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile33930.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile33980.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile34030.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/Bedroom/outfile34080.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2

...

C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile138114.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile138614.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile139114.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile139614.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile140114.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile140614.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile141114.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile14114.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile141614.jpg
C:

C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile227714.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile228214.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile228714.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile229214.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile229714.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile230214.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile230714.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile231214.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile231714.jpg
C

C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile322314.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile322814.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile32314.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile323314.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile323814.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile324314.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile324814.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile325314.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile325814.jpg
C:

C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile414414.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile414914.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile415414.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile415914.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile416414.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile416914.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile417414.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile417914.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile418414.jpg
C

C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile46914.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile47414.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile47914.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile48414.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile48914.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile4914.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile49414.jpg
C:\Users\39320\Desktop\computer vision\Computer-Vision-2020-main\ImageSet\train/TallBuilding/outfile49914.jpg


In [ ]:
testImages, featureMatrixTest, featureArrayTest = extractFeatures(model, pathTest, labels)

In [ ]:
classif = [SVC(kernel="linear") for _ in range(numberOfClasses)]

currentLabel = 0
for clf in classif:
    print(currentLabel)
    v = np.array([1 if label==currentLabel else 0 for label in yTrain])
    clf = clf.fit(featureArray, v)
    currentLabel += 1

In [ ]:
prediction = []
for image in featureArrayTest:
    pred = np.array([np.dot(clf.coef_,image) + clf.intercept_ for clf in classif])
    prediction.append(np.argmax(pred))
    
prediction = np.asarray(prediction)

In [ ]:
cont=0
for i in range(len(prediction)):
    if prediction[i]==yTest[i]:
        cont += 1
    else:
        continue
        
accuracy = cont/len(prediction)
print("Accuracy value: ",accuracy)

In [ ]:
tp = []
fp = []
fn = []
tn = []
for i in range(numberOfClasses):
    tp_temp = 0
    fp_temp = 0
    fn_temp = 0
    tn_temp = 0
    
    for j in range(len(prediction)):
        if(prediction[j]==i and yTest[j]==i):
            tp_temp += 1
        if(prediction[j]==i and yTest[j]!=i):
            fp_temp += 1
        if(prediction[j]!=i and yTest[j]==i):
            fn_temp += 1
        if(prediction[j]!=i and yTest[j]!=i):
            tn_temp += 1
    tp.append(tp_temp)
    fp.append(fp_temp)
    fn.append(fn_temp)
    tn.append(tn_temp)

In [ ]:
data = {'labels':labels , 'True positive':tp,'True negative':tn,'False positive':fp,'False negative':fn}
evaluation = pd.DataFrame(data, columns = ['labels','True positive','True negative','False positive','False negative'])
print("Evaluation: ")
evaluation

In [ ]:
cm = np.zeros((15,15))
for i in range(numberOfClasses):
    cm[i,i] = evaluation.loc[i]['True positive']
    
for i in range(numberOfClasses):
    for j in range(numberOfClasses):
        temp = 0
        for k in range(len(yTest)):
            if(yTest[k]==i and prediction[k]==j):
                temp += 1
        cm[i,j]=temp

In [ ]:
pltu.plotConfusionMatrix(cm, labels, "confusion_matrix_2", "images_point_three/")